In [1]:
from credit.data import get_forward_data
from glob import glob
import os
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from os.path import join
from credit.xr_sampler import XRSamplerByYear
import yaml


In [2]:
# rollout_dir = "/glade/derecho/scratch/dkimpara/CREDIT_runs/skebs_diss_train"
rollout_dir = "/glade/derecho/scratch/dkimpara/CREDIT_runs/camulator_rollout"
# rollout_dir = "/glade/derecho/scratch/dkimpara/CREDIT_runs/skebs_fcnn_scaled_train"
# rollout_dir = "/glade/derecho/scratch/dkimpara/CREDIT_runs/archive_skebs/skebs_conv_train"

In [3]:
with open("/glade/derecho/scratch/dkimpara/CREDIT_runs/skebs_diss_train/model.yml") as cf:
    conf = yaml.load(cf, Loader=yaml.FullLoader)

sampler = XRSamplerByYear("/glade/derecho/scratch/dkimpara/CREDIT_runs/skebs_diss_train/model.yml")

def get_data(rollout_files):
    # get pred and true data
    pred_da_list = []
    true_da_list = []
    for file in rollout_files:
        ds = xr.open_dataset(file)
        pred_da_list.append(ds)
        
        da_true = sampler(ds.time.values[0])
        true_da_list.append(da_true)
    
    da_pred = xr.concat(pred_da_list, dim='time')
    da_true = xr.concat(true_da_list, dim='time')
    da_true = da_true.assign_coords(level=da_pred.level)
    return da_true, da_pred

## define metrics

### spread-error



In [4]:
def error_spread(file):
    # rollout_files = glob(join(rollout_dir, f"*{eval_fh:03}.nc"))
    obs, pred = get_data([file])

    if "ensemble_member_label" in pred.coords:
        std = pred.std(dim="ensemble_member_label")
        pred = pred.mean(dim="ensemble_member_label")
        error = np.sqrt((obs - pred) ** 2) # * lat_wts / sum_wts

        std.to_netcdf(join(os.path.dirname(file), f"std_{os.path.basename(file)}"))
        error.to_netcdf(join(os.path.dirname(file), f"err_{os.path.basename(file)}"))
        
        return error, std

    error = np.sqrt((obs - pred) ** 2) # * lat_wts / sum_wts
    error.to_netcdf(join(os.path.dirname(file), f"err_{os.path.basename(file)}"))

    return error

In [10]:
eval_fhs = range(6, 241, 6)

rollout_files = []
for eval_fh in eval_fhs:
    rollout_files += glob(join(rollout_dir, f"2012*/pred*{eval_fh:03}.nc")) + glob(join(rollout_dir, f"2013*/pred*{eval_fh:03}.nc"))

# run metrics

In [7]:
import multiprocessing
from functools import partial

In [11]:
with multiprocessing.Pool(7) as p:
    # f = partial(error_spread, mode_ensemble=mode_ensemble)
    f = error_spread
    result = p.map(f, rollout_files)

# visualize rmse

In [13]:
err_files = [join(os.path.dirname(file), f"err_{os.path.basename(file)}") for file in rollout_files]

In [ ]:
err_files = err_files[:2